In [1]:
# Basic imports
import scipy, pandas as pd, numpy as np
from matplotlib import pyplot as plt

In [66]:
# Zone to FIPS crosswalk
ztofips =pd.read_csv("ZonetoFIPS.txt", names=["STATE", "ZONE", "CWA", "NAME", "STATE_ZONE", "COUNTY", "FIPS", "TIME_ZONE", "FE_AREA", "LAT", "LON"], sep='|')
ztofips

,STATE,ZONE,CWA,NAME,STATE_ZONE,COUNTY,FIPS,TIME_ZONE,FE_AREA,LAT,LON
0,NM,201,ABQ,Northwest Plateau,NM201,McKinley,35031,M,nw,36.4270,-108.4064
1,NM,201,ABQ,Northwest Plateau,NM201,San Juan,35045,M,nw,36.4270,-108.4064
2,NM,202,ABQ,Chuska Mountains,NM202,McKinley,35031,M,nw,36.0409,-108.9296
3,NM,202,ABQ,Chuska Mountains,NM202,San Juan,35045,M,nw,36.0409,-108.9296
4,NM,203,ABQ,Far Northwest Highlands,NM203,Rio Arriba,35039,M,nc,36.6585,-107.1028
...,...,...,...,...,...,...,...,...,...,...,...
4788,WA,209,PQR,West Columbia River Gorge - SR 14,WA209,Skamania,53059,P,sw,45.6379,-122.0056
4789,WA,210,PQR,Central Columbia River Gorge - SR 14,WA210,Skamania,53059,P,sw,45.7152,-121.7109
4790,WA,211,PQR,South Washington Cascades,WA211,Skamania,53059,P,sw,46.0559,-122.0135
4791,WA,211,PQR,South Washington Cascades,WA211,Cowlitz,53015,P,sw,46.0559,-122.0135


In [41]:
# state abbreviations and fips crosswalk
fipstoab = pd.read_csv("us-state-ansi-fips.csv")

In [65]:
fipstoab.rename(columns={' st': 'STATE_FIPS'}, inplace=True)

In [43]:
# look at the noloc files
noloc06 = pd.read_csv("storm_noloc/storm2006_noloc.csv")
noloc06

,OID_,F_,STATE_FIPS,CZ_FIPS,CZ_TYPE,EVENT_TYPE,BEGIN_YEARMONTH,BEGIN_DAY,END_YEARMONTH,END_DAY,LATITUDE,LONGITUDE,beg_datetime,days
0,1,5482463,8,2,Z,Drought,200601,1,200601,31,nan,nan,1/1/2006 0:00:00,31
1,2,5482464,8,7,Z,Drought,200601,1,200601,31,nan,nan,1/1/2006 0:00:00,31
2,3,5482465,8,4,Z,Drought,200601,1,200601,31,nan,nan,1/1/2006 0:00:00,31
3,4,5482466,8,13,Z,Drought,200601,1,200601,31,nan,nan,1/1/2006 0:00:00,31
4,5,5482480,49,23,Z,Drought,200601,1,200601,31,nan,nan,1/1/2006 0:00:00,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6390,6391,2184,44,5,Z,Coastal Flood,200610,28,200610,28,nan,nan,10/28/2006 0:00:00,1
6391,6392,2239,40,53,Z,Drought,200610,1,200610,31,nan,nan,10/1/2006 0:00:00,31
6392,6393,2255,40,69,Z,Drought,200610,1,200610,31,nan,nan,10/1/2006 0:00:00,31
6393,6394,2258,40,72,Z,Drought,200610,1,200610,31,nan,nan,10/1/2006 0:00:00,31


In [44]:
# conver the datetime file back to datetime object from text
noloc06['beg_datetime'] = pd.to_datetime(noloc06['beg_datetime'], format='%m/%d/%Y %H:%M:%S')

In [45]:
noloc06.dtypes

OID_                        int64
F_                          int64
STATE_FIPS                  int64
CZ_FIPS                     int64
CZ_TYPE                    object
EVENT_TYPE                 object
BEGIN_YEARMONTH             int64
BEGIN_DAY                   int64
END_YEARMONTH               int64
END_DAY                     int64
LATITUDE                   object
LONGITUDE                  object
beg_datetime       datetime64[ns]
days                        int64
dtype: object

In [46]:
noloc06['CZ_TYPE'].unique()

array([' Z', ' C'], dtype=object)

In [47]:
zones06 = noloc06[noloc06['CZ_TYPE'] == ' Z']

In [48]:
zones06

,OID_,F_,STATE_FIPS,CZ_FIPS,CZ_TYPE,EVENT_TYPE,BEGIN_YEARMONTH,BEGIN_DAY,END_YEARMONTH,END_DAY,LATITUDE,LONGITUDE,beg_datetime,days
0,1,5482463,8,2,Z,Drought,200601,1,200601,31,nan,nan,2006-01-01,31
1,2,5482464,8,7,Z,Drought,200601,1,200601,31,nan,nan,2006-01-01,31
2,3,5482465,8,4,Z,Drought,200601,1,200601,31,nan,nan,2006-01-01,31
3,4,5482466,8,13,Z,Drought,200601,1,200601,31,nan,nan,2006-01-01,31
4,5,5482480,49,23,Z,Drought,200601,1,200601,31,nan,nan,2006-01-01,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6390,6391,2184,44,5,Z,Coastal Flood,200610,28,200610,28,nan,nan,2006-10-28,1
6391,6392,2239,40,53,Z,Drought,200610,1,200610,31,nan,nan,2006-10-01,31
6392,6393,2255,40,69,Z,Drought,200610,1,200610,31,nan,nan,2006-10-01,31
6393,6394,2258,40,72,Z,Drought,200610,1,200610,31,nan,nan,2006-10-01,31


In [50]:
zones06_ab = pd.merge(zones06, fipstoab, on='STATE_FIPS', how='left')

In [62]:
zones06_ab
na_mask = zones06_ab[' stusps'].isna()
zones06_ab[na_mask]

,OID_,F_,STATE_FIPS,CZ_FIPS,CZ_TYPE,EVENT_TYPE,BEGIN_YEARMONTH,BEGIN_DAY,END_YEARMONTH,END_DAY,LATITUDE,LONGITUDE,beg_datetime,days,stname,stusps
250,275,5493196,99,8,Z,Coastal Flood,200601,16,200601,19,nan,nan,2006-01-16,4,NaN,NaN
867,1013,5493206,99,10,Z,Coastal Flood,200601,17,200601,17,nan,nan,2006-01-17,1,NaN,NaN
1516,2799,5521314,98,2,Z,Tropical Storm,200607,1,200607,1,nan,nan,2006-07-01,1,NaN,NaN
2068,4379,5531607,98,5,Z,Tropical Storm,200608,5,200608,6,nan,nan,2006-08-05,2,NaN,NaN
2069,4380,5531608,98,6,Z,Tropical Storm,200608,5,200608,6,nan,nan,2006-08-05,2,NaN,NaN
2255,4795,5534241,97,3,Z,Tsunami,200609,28,200609,28,nan,nan,2006-09-28,1,NaN,NaN
2396,5115,5534219,97,2,Z,Tsunami,200609,28,200609,28,nan,nan,2006-09-28,1,NaN,NaN
2961,6086,8029,97,2,Z,Tsunami,200611,15,200611,15,nan,nan,2006-11-15,1,NaN,NaN
3043,6168,8032,98,5,Z,Tropical Storm,200610,11,200610,12,nan,nan,2006-10-11,2,NaN,NaN


In [64]:
notna_mask = zones06_ab[' stusps'].notna()
zones06_ab[notna_mask]

,OID_,F_,STATE_FIPS,CZ_FIPS,CZ_TYPE,EVENT_TYPE,BEGIN_YEARMONTH,BEGIN_DAY,END_YEARMONTH,END_DAY,LATITUDE,LONGITUDE,beg_datetime,days,stname,stusps
0,1,5482463,8,2,Z,Drought,200601,1,200601,31,nan,nan,2006-01-01,31,Colorado,CO
1,2,5482464,8,7,Z,Drought,200601,1,200601,31,nan,nan,2006-01-01,31,Colorado,CO
2,3,5482465,8,4,Z,Drought,200601,1,200601,31,nan,nan,2006-01-01,31,Colorado,CO
3,4,5482466,8,13,Z,Drought,200601,1,200601,31,nan,nan,2006-01-01,31,Colorado,CO
4,5,5482480,49,23,Z,Drought,200601,1,200601,31,nan,nan,2006-01-01,31,Utah,UT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3266,6391,2184,44,5,Z,Coastal Flood,200610,28,200610,28,nan,nan,2006-10-28,1,Rhode Island,RI
3267,6392,2239,40,53,Z,Drought,200610,1,200610,31,nan,nan,2006-10-01,31,Oklahoma,OK
3268,6393,2255,40,69,Z,Drought,200610,1,200610,31,nan,nan,2006-10-01,31,Oklahoma,OK
3269,6394,2258,40,72,Z,Drought,200610,1,200610,31,nan,nan,2006-10-01,31,Oklahoma,OK
